In [1]:
import os
import sys
import time
os.environ['TF_ENABLE_ONEDNN_OPTS'] = str(0)

import numpy as np
import pandas as pd
import sklearn
import tensorflow as tf
from tensorflow import keras  # tf.keras
import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
print("python", sys.version)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

python 3.10.9 | packaged by Anaconda, Inc. | (main, Mar  1 2023, 18:18:15) [MSC v.1916 64 bit (AMD64)]
matplotlib 3.7.0
numpy 1.23.5
pandas 1.5.3
sklearn 1.2.1
tensorflow 2.12.0
keras.api._v2.keras 2.12.0


In [3]:
assert sys.version_info >= (3, 5) # Python ≥3.5 required
assert tf.__version__ >= "2.0"    # TensorFlow ≥2.0 required

In [4]:
from test_utility import *

# Start Test

In [5]:
n_times = 1000; n_markets = 100; n_fields = 10
Candles = [ [ [ time * n_markets * n_fields + market * n_fields + field for field in range(n_fields) ] for market in range(n_markets) ] for time in range(n_times)]
Candles = np.array(Candles, dtype=np.float32)
Times = np.array( range(Candles.shape[0]), dtype=Candles.dtype ) + 333
Times = np.expand_dims(Times, axis=1)
size_time = 1
print(Candles.shape, Times.shape)   # time, market, field
print(Candles[:2, :, :])

(1000, 100, 10) (1000, 1)
[[[0.000e+00 1.000e+00 2.000e+00 ... 7.000e+00 8.000e+00 9.000e+00]
  [1.000e+01 1.100e+01 1.200e+01 ... 1.700e+01 1.800e+01 1.900e+01]
  [2.000e+01 2.100e+01 2.200e+01 ... 2.700e+01 2.800e+01 2.900e+01]
  ...
  [9.700e+02 9.710e+02 9.720e+02 ... 9.770e+02 9.780e+02 9.790e+02]
  [9.800e+02 9.810e+02 9.820e+02 ... 9.870e+02 9.880e+02 9.890e+02]
  [9.900e+02 9.910e+02 9.920e+02 ... 9.970e+02 9.980e+02 9.990e+02]]

 [[1.000e+03 1.001e+03 1.002e+03 ... 1.007e+03 1.008e+03 1.009e+03]
  [1.010e+03 1.011e+03 1.012e+03 ... 1.017e+03 1.018e+03 1.019e+03]
  [1.020e+03 1.021e+03 1.022e+03 ... 1.027e+03 1.028e+03 1.029e+03]
  ...
  [1.970e+03 1.971e+03 1.972e+03 ... 1.977e+03 1.978e+03 1.979e+03]
  [1.980e+03 1.981e+03 1.982e+03 ... 1.987e+03 1.988e+03 1.989e+03]
  [1.990e+03 1.991e+03 1.992e+03 ... 1.997e+03 1.998e+03 1.999e+03]]]


In [6]:
Nx = 10
Ny = 2
Ns = 10

sample_anchors = np.array(range(0, Candles.shape[0] - Nx - Ny + 1, Ns), dtype=np.int32)
sample_anchores_t = sample_anchors[: sample_anchors.shape[0] * 3 // 4]
sample_anchores_v = sample_anchors[sample_anchors.shape[0] * 3 // 4 :]
print(Candles.shape, sample_anchors.shape, sample_anchores_t.shape, sample_anchores_v.shape)

x_indices = ( (0, 1, 2), (0,1,2) )    # (market, field)
y_indices = ( (2, 3, 4), (1,2,3) )    # (market, field). Currently must be the same size as x_indices.
print(Candles[0:2][:, x_indices[0]][:, :, x_indices[1]])
print(Candles[2:4][:, y_indices[0]][:, :, y_indices[1]])

size_x = get_timepoint_size(x_indices)
size_y = get_timepoint_size(y_indices)

print(size_x, size_y, size_time)

(1000, 100, 10) (99,) (74,) (25,)
[[[0.000e+00 1.000e+00 2.000e+00]
  [1.000e+01 1.100e+01 1.200e+01]
  [2.000e+01 2.100e+01 2.200e+01]]

 [[1.000e+03 1.001e+03 1.002e+03]
  [1.010e+03 1.011e+03 1.012e+03]
  [1.020e+03 1.021e+03 1.022e+03]]]
[[[2021. 2022. 2023.]
  [2031. 2032. 2033.]
  [2041. 2042. 2043.]]

 [[3021. 3022. 3023.]
  [3031. 3032. 3033.]
  [3041. 3042. 3043.]]]
9 9 1


In [7]:
# Dummy variables
CandleMarks = Candles[:, :, 0]
all_market_names = [ 'market_' + str(i) for i in range(Candles.shape[1])]
all_field_names = [ 'filed_' + str(i) for i in range(Candles.shape[2])]
min_true_candle_percent_x = 50
chosen_fields_names_x = all_field_names[2:7]
min_true_candle_percent_y = 50
chosen_fields_names_y = all_field_names[0:5]
target_market_names = all_market_names[2:15] #[0:1] # [1:2]
tarket_market_top_percent = 15

Candles, CandleMarks, all_market_names, x_indices, y_indices, \
chosen_market_names_x, chosen_field_names_x, chosen_market_names_y, chosen_field_names_y, \
chosen_market_names, chosen_field_names, \
target_markets_names, target_markets = \
get_formed_data( Candles, CandleMarks, all_market_names, all_field_names, 
        min_true_candle_percent_x, chosen_fields_names_x, min_true_candle_percent_y, chosen_fields_names_y,
        target_market_names, tarket_market_top_percent
)

print(Candles.shape)
print(CandleMarks.shape)
print(len(all_market_names))
print(x_indices)
print(y_indices)
print(chosen_market_names_x)
print(chosen_field_names_x)
print(chosen_market_names_y)
print(chosen_field_names_y)
print(chosen_market_names)
print(chosen_field_names)
print(target_markets_names)
print(target_markets)
print(len(chosen_market_names_x), len(chosen_market_names_y), len(target_markets_names))

(1000, 100, 7)
(1000, 100)
100
((0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99), (2, 3, 4, 5, 6))
((0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99), (0, 1, 2, 3, 4))
['market_49', 'market_72', 'market_71', 'market_70', 'market_69', 'market_68', 'market_67', 'market_66', 'market_65', 'market_64', 'market_63', 'mark

In [8]:
Time_into_X = True
Time_into_Y = False
BatchSize = 2
shuffle_batch = 10

ds_train, ds_valid, dx, dy = \
get_datasets_2(
    Candles, Time_into_X, Time_into_Y, Times, 
    sample_anchores_t, sample_anchores_v,
    Nx, x_indices, Ny, y_indices, size_time, target_markets,
    BatchSize, shuffle_batch, shuffle=True
)

print(len(ds_train), len(ds_valid))

37 13


In [9]:
non_target_markets_relative = [ y_indices[0].index(i) for i in y_indices[0] if i not in target_markets ]
print(non_target_markets_relative)
nFeaturesPerMarket = len(y_indices[1])
print(nFeaturesPerMarket)
all_ntmr = []
for ntmr in non_target_markets_relative:
    all_ntmr = all_ntmr + list(range(ntmr* nFeaturesPerMarket, (ntmr+1) * nFeaturesPerMarket, 1))
non_target_features_relative = tuple(all_ntmr)
print(non_target_features_relative)

def anchor_to_sample(anchor): # This function is the bottle-neck of training speed.
    x = np.reshape(Candles[anchor: anchor + Nx][:, x_indices[0]][:, :, x_indices[1]], (Nx, -1))
    if Time_into_X is True:
        assert Times is not None
        x = np.concatenate((x, np.reshape(Times[anchor: anchor + Nx], (Nx, -1))), axis=1) # concat(x, x_time)

    print('x.shape', x.shape, x_indices)

    y = np.reshape(Candles[anchor + Nx: anchor + Nx + Ny][:, y_indices[0]][:, :, y_indices[1]], (Ny, -1))
    print('y.shape', y.shape, y_indices)

    if Time_into_X is True:
        assert Times is not None
        y_time = np.reshape(Times[anchor + Nx: anchor + Nx + Ny], (Ny, -1))
        if Time_into_Y is not True: y_time[:] = 0.0
        y = np.concatenate((y, y_time), axis=1)

    x = np.pad(x, [[1,1], [0,0]], constant_values=0)   # (1 pre-pad: Start, 1 post-pad: End) on axis 0. (0 pre-pad, 0 post-pad) on axis 1.
    z = y
    y = np.pad(y, [[1,1], [0,0]], constant_values=0)

    if x.shape[-1] % 2 != 0:
        x = np.pad(x, [[0,0], [0,1]], constant_values=0) # (0 pre-pad: Start, 0 post-pad: End) on axis 0. (0 pre-pad, 1 post-pad) on axis 1.
        y = np.pad(y, [[0,0], [0,1]], constant_values=0)

    y_target = np.copy(y[1:])
    print(y_target.shape)
    y_target[:, non_target_features_relative] = 0.0

    return x, y[:-1], y_target

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
5
(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153

In [10]:
print(Candles[:4, :, :]) # market order may be changed by argmax.

[[[4.900e+02 4.910e+02 4.920e+02 ... 4.940e+02 4.950e+02 4.960e+02]
  [7.200e+02 7.210e+02 7.220e+02 ... 7.240e+02 7.250e+02 7.260e+02]
  [7.100e+02 7.110e+02 7.120e+02 ... 7.140e+02 7.150e+02 7.160e+02]
  ...
  [2.600e+02 2.610e+02 2.620e+02 ... 2.640e+02 2.650e+02 2.660e+02]
  [3.600e+02 3.610e+02 3.620e+02 ... 3.640e+02 3.650e+02 3.660e+02]
  [0.000e+00 1.000e+00 2.000e+00 ... 4.000e+00 5.000e+00 6.000e+00]]

 [[1.490e+03 1.491e+03 1.492e+03 ... 1.494e+03 1.495e+03 1.496e+03]
  [1.720e+03 1.721e+03 1.722e+03 ... 1.724e+03 1.725e+03 1.726e+03]
  [1.710e+03 1.711e+03 1.712e+03 ... 1.714e+03 1.715e+03 1.716e+03]
  ...
  [1.260e+03 1.261e+03 1.262e+03 ... 1.264e+03 1.265e+03 1.266e+03]
  [1.360e+03 1.361e+03 1.362e+03 ... 1.364e+03 1.365e+03 1.366e+03]
  [1.000e+03 1.001e+03 1.002e+03 ... 1.004e+03 1.005e+03 1.006e+03]]

 [[2.490e+03 2.491e+03 2.492e+03 ... 2.494e+03 2.495e+03 2.496e+03]
  [2.720e+03 2.721e+03 2.722e+03 ... 2.724e+03 2.725e+03 2.726e+03]
  [2.710e+03 2.711e+03 2.712e+03

In [11]:
x, y, y_shift = anchor_to_sample(0)
print(x.shape)
print(y.shape)
print(y_shift.shape)

x.shape (10, 501) ((0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99), (2, 3, 4, 5, 6))
y.shape (2, 500) ((0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99), (0, 1, 2, 3, 4))
(3, 502)
(12, 502)
(3, 502)
(3, 502)


In [12]:
assert x.shape[0] == Nx + 2

seq = 0
assert np.sum(x[seq]>0) == 0

seq = 1
batch = len(chosen_fields_names_x)
for b in range(0, x.shape[1], batch):
    print(x[seq][b: b + batch])
assert x[seq][-1] == 0.0     # -1 for the odd-padding
assert x[seq][-2] > 0     # time

seq = 2
batch = len(chosen_fields_names_x)
for b in range(0, x.shape[1], batch):
    print(x[seq][b: b + batch])
assert x[seq][-1] == 0.0     # -1 for the odd-padding
assert x[seq][-2] > 0     # time


[492. 493. 494. 495. 496.]
[722. 723. 724. 725. 726.]
[712. 713. 714. 715. 716.]
[702. 703. 704. 705. 706.]
[692. 693. 694. 695. 696.]
[682. 683. 684. 685. 686.]
[672. 673. 674. 675. 676.]
[662. 663. 664. 665. 666.]
[652. 653. 654. 655. 656.]
[642. 643. 644. 645. 646.]
[632. 633. 634. 635. 636.]
[622. 623. 624. 625. 626.]
[612. 613. 614. 615. 616.]
[602. 603. 604. 605. 606.]
[592. 593. 594. 595. 596.]
[582. 583. 584. 585. 586.]
[572. 573. 574. 575. 576.]
[562. 563. 564. 565. 566.]
[552. 553. 554. 555. 556.]
[542. 543. 544. 545. 546.]
[532. 533. 534. 535. 536.]
[522. 523. 524. 525. 526.]
[732. 733. 734. 735. 736.]
[512. 513. 514. 515. 516.]
[742. 743. 744. 745. 746.]
[762. 763. 764. 765. 766.]
[972. 973. 974. 975. 976.]
[962. 963. 964. 965. 966.]
[952. 953. 954. 955. 956.]
[942. 943. 944. 945. 946.]
[932. 933. 934. 935. 936.]
[922. 923. 924. 925. 926.]
[912. 913. 914. 915. 916.]
[902. 903. 904. 905. 906.]
[892. 893. 894. 895. 896.]
[882. 883. 884. 885. 886.]
[872. 873. 874. 875. 876.]
[

In [13]:
assert y.shape[0] == Ny + 1

seq = 0
assert np.sum(y[seq]>0) == 0

seq = 1
batch = len(chosen_fields_names_y)
for b in range(0, y.shape[1], batch):
    print(y[seq][b: b + batch])
assert y[seq][-1] == 0.0     # -1 for the odd-padding
assert y[seq][-2] == 0.0     # Time_into_y: False
assert np.sum(y[seq]>0) != len(target_markets_names) * len(chosen_field_names_y)

seq = 2
batch = len(chosen_fields_names_y)
for b in range(0, y.shape[1], batch):
    print(y[seq][b: b + batch])
assert y[seq][-1] == 0.0    # -1 for the odd-padding
assert y[seq][-2] == 0.0    # Time_into_y: False
assert np.sum(y[seq]>0) != len(target_markets_names) * len(chosen_field_names_y)

[10490. 10491. 10492. 10493. 10494.]
[10720. 10721. 10722. 10723. 10724.]
[10710. 10711. 10712. 10713. 10714.]
[10700. 10701. 10702. 10703. 10704.]
[10690. 10691. 10692. 10693. 10694.]
[10680. 10681. 10682. 10683. 10684.]
[10670. 10671. 10672. 10673. 10674.]
[10660. 10661. 10662. 10663. 10664.]
[10650. 10651. 10652. 10653. 10654.]
[10640. 10641. 10642. 10643. 10644.]
[10630. 10631. 10632. 10633. 10634.]
[10620. 10621. 10622. 10623. 10624.]
[10610. 10611. 10612. 10613. 10614.]
[10600. 10601. 10602. 10603. 10604.]
[10590. 10591. 10592. 10593. 10594.]
[10580. 10581. 10582. 10583. 10584.]
[10570. 10571. 10572. 10573. 10574.]
[10560. 10561. 10562. 10563. 10564.]
[10550. 10551. 10552. 10553. 10554.]
[10540. 10541. 10542. 10543. 10544.]
[10530. 10531. 10532. 10533. 10534.]
[10520. 10521. 10522. 10523. 10524.]
[10730. 10731. 10732. 10733. 10734.]
[10510. 10511. 10512. 10513. 10514.]
[10740. 10741. 10742. 10743. 10744.]
[10760. 10761. 10762. 10763. 10764.]
[10970. 10971. 10972. 10973. 10974.]
[

In [14]:
assert y_shift.shape[0] == Ny + 1

seq = 0
batch = len(chosen_fields_names_y)
for b in range(0, y_shift.shape[1], batch):
    print(y_shift[seq][b: b + batch])
assert y_shift[seq][-1] == 0.0     # -1 for the odd-padding
assert y_shift[seq][-2] == 0.0     # Time_into_y: False
assert np.sum(y_shift[seq]>0) == len(target_markets_names) * len(chosen_field_names_y)

seq = 1
batch = len(chosen_fields_names_y)
for b in range(0, y.shape[1], batch):
    print(y_shift[seq][b: b + batch])
assert y_shift[seq][-1] == 0.0    # -1 for the odd-padding
assert y_shift[seq][-2] == 0.0    # Time_into_y: False
assert np.sum(y_shift[seq]>0) == len(target_markets_names) * len(chosen_field_names_y)

seq = 2
assert np.sum(y_shift[seq]>0) == 0

[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[10140. 10141.

In [15]:
ds_train, ds_valid, dx, dy = \
get_datasets_2(
    Candles, Time_into_X, Time_into_Y, Times, 
    sample_anchores_t, sample_anchores_v,
    Nx, x_indices, Ny, y_indices, size_time, target_markets,
    BatchSize, shuffle_batch, shuffle=True
)

In [16]:
for elem in ds_train:
    print(elem)
    break

((<tf.Tensor: shape=(2, 12, 502), dtype=float32, numpy=
array([[[    0.,     0.,     0., ...,     0.,     0.,     0.],
        [20492., 20493., 20494., ..., 20006.,   353.,     0.],
        [21492., 21493., 21494., ..., 21006.,   354.,     0.],
        ...,
        [28492., 28493., 28494., ..., 28006.,   361.,     0.],
        [29492., 29493., 29494., ..., 29006.,   362.,     0.],
        [    0.,     0.,     0., ...,     0.,     0.,     0.]],

       [[    0.,     0.,     0., ...,     0.,     0.,     0.],
        [80492., 80493., 80494., ..., 80006.,   413.,     0.],
        [81492., 81493., 81494., ..., 81006.,   414.,     0.],
        ...,
        [88492., 88493., 88494., ..., 88006.,   421.,     0.],
        [89492., 89493., 89494., ..., 89006.,   422.,     0.],
        [    0.,     0.,     0., ...,     0.,     0.,     0.]]],
      dtype=float32)>, <tf.Tensor: shape=(2, 3, 502), dtype=float32, numpy=
array([[[    0.,     0.,     0., ...,     0.,     0.,     0.],
        [30490., 30

# End Test

In [ ]:
from test_utility import *
# from test_trans import *

In [ ]:
dir_data = "/mnt/data/Trading/"

data_model = "mv03.200.5.60.15.12.1"

#===================================================================== Dataset

Nx = 200 # ------------- test
Ny = 5
Ns = 5 #--------------------- test
BatchSize = 64

CandleFile = "18-01-01-00-00-23-05-20-20-23-5m"
SmallSigma = 1
LargeSigma = 30
eFreeNoLog = True

shuffle_batch = 50  # Keep it small to speed up model loading.

dir_candles = os.path.join(dir_data, "Candles")

min_true_candle_percent_x = 60
chosen_markets_x = []
chosen_fields_names_x = ['ClosePrice'] #, 'BaseVolume']
min_true_candle_percent_y = 60
assert min_true_candle_percent_x == min_true_candle_percent_y
chosen_markets_y = []
chosen_fields_names_y = ['ClosePrice']

target_market_names = None
# target_market_names = ['NEOUSDT', 'LTCUSDT', 'BTCUSDT', 'ETHUSDT', 'BNBUSDT', 'QTUMUSDT', 'ADAUSDT', 'XRPUSDT']
tarket_market_top_percent = 15

Standardization = True
Kill_Irregulars = True  # ----------------- pls implement it
Time_into_X = True
Time_into_Y = False #
eFreeNoPlot = True

#======================================================================== Model

Num_Layers = 12 # Wow
Num_Heads = 1   # As we have a single GPU, and we want to a exhaustic attention.
Factor_FF = 4
repComplexity = 12  # Wower
Dropout_Rate = 0.1

dir_Checkpoint = os.path.join(dir_data, "Checkpoints")
checkpoint_filepath = os.path.join(dir_Checkpoint, data_model)
dir_CSVLogs = os.path.join(dir_data, "CSVLogs")
csvLogger_filepath = os.path.join(dir_CSVLogs, data_model)

#======================================================================== Train

Epochs_Initial = 5000
HuberThreshold = 1.0
Checkpoint_Monitor = "val_loss"
EarlyStopping_Min_Monitor = "val_loss"
EarlyStopping_Patience = 30

Optimizer = "adam"
Learning_Rate = 0.0001  # default: 0.001

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    print(gpus)
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            # tf.config.experimental.set_virtual_device_configuration(
            #     gpu,[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5120)]) # why 5120?
            # logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            # print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

mirrored_strategy = None
if len(gpus) > 1: 
    mirrored_strategy = tf.distribute.MirroredStrategy()
    Learning_Rate = Learning_Rate * len(gpus) * 3 / 4

# tf.config.experimental.set_virtual_device_configuration(
#     gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5120)])

In [ ]:
from datetime import datetime, timedelta
import sys 
sys.path.append('..')
import json

In [ ]:
def upgrade_file(path):
    with open(path, "+tw") as f:
        f.write("# Sorry, the content is removed.")
        f.write("\n# Please ask Mike for the content.")

In [ ]:
folders = [dir_data, dir_candles, dir_Checkpoint, dir_CSVLogs]
for folder in folders:
    if not os.path.isdir(folder):
        os.mkdir(folder)

In [ ]:
#==================== Load candle data into 'table' with shape of (time, markets, 10 fields) ====================
Candles = np.load( os.path.join( dir_candles, "table-" + CandleFile + ".npy") )
Candles = np.swapaxes(Candles, 0, 1)
Candles = Candles.astype(np.float32)    # Ugly, just confirm.
print("Candles: {}".format(Candles.shape))

In [ ]:
market = 5
Show_Price_Volume_10(Candles[:, market, :], 1, 1, 500)

In [ ]:
Event_Free_Learning_Scheme_10(Candles[:, market, :], 3, 30, 500)

In [ ]:
#==================== Delete 7 candle fields from 'Candles'. ====================
# Candles.shape becomes (time, markets, ['ClosePrice', 'BaseVolume', 'BuyerBaseVolume'] )

CandleMarks = Candles[:, :, 9] # keep it for later use
Candles = np.delete(Candles, [0, 1, 2, 5, 6, 8, 9], axis = 2) # delete Open, High, Low, qVolume, #Trades, bQVolume, CandleMarks
all_field_names = ['ClosePrice', 'BaseVolume', 'BuyerBaseVolume']

assert (~np.isfinite(Candles)).any() == False

table_markets = []
with open( os.path.join( dir_candles, "reports-" + CandleFile + ".json"), "r") as f:
    reports = json.loads(f.read())
print(reports[:2])

all_market_names = [ s[0: s.find(':')] for s in reports if 'Success' in s ]
assert Candles.shape[1] == len(all_market_names)
print(Candles.shape, len(all_market_names), all_market_names[:2])

In [ ]:
Candles, CandleMarks, all_market_names, x_indices, y_indices, \
chosen_market_names_x, chosen_field_names_x, chosen_market_names_y, chosen_field_names_y, \
chosen_market_names, chosen_field_names, \
target_markets_names, target_markets = \
get_formed_data( Candles, CandleMarks, all_market_names, all_field_names, 
        min_true_candle_percent_x, chosen_fields_names_x, min_true_candle_percent_y, chosen_fields_names_y,
        target_market_names, tarket_market_top_percent
)

print(Candles.shape)
print(CandleMarks.shape)
print(len(all_market_names))
print(x_indices)
print(y_indices)
print(chosen_market_names_x)
print(chosen_field_names_x)
print(chosen_market_names_y)
print(chosen_field_names_y)
print(chosen_market_names)
print(chosen_field_names)
print(target_markets_names)
print(target_markets)
print(len(chosen_market_names_x), len(chosen_market_names_y), len(target_markets_names))

In [ ]:
start_ts, interval_s, timestamps_abs = get_timestamps_2(CandleFile, Candles.shape[0])
print(start_ts, interval_s, timestamps_abs.shape, timestamps_abs[:3])

Times = get_time_features(timestamps_abs)
Times = Times.astype(Candles.dtype)
size_time = Times.shape[1]

assert Candles.shape[0] == Times.shape[0]
print(Candles.shape, Times.shape)

In [ ]:
#==================== Generate event-free data into Data ====================
# Data loses heading items.
# Do it before: Permute Data in time

alpha = 3; beta = 3 # beta is used in 'get_eFree_with_plot'. Ugly coupling.
event_free_data_loss = 3 * ( alpha * SmallSigma + LargeSigma)
eFree = np.zeros( (Candles.shape[0] - event_free_data_loss, len(chosen_market_names), len(chosen_field_names)), dtype = Candles.dtype )

for market in range(Candles.shape[1]):
    for field in range(Candles.shape[2]):
        sSigma = SmallSigma
        if all_field_names[field] == 'BaseVolume': sSigma = SmallSigma * alpha
        P, maP, logP, log_maP, event, eventFree = \
        get_eFree_with_plot(all_market_names[market], all_field_names[field], Candles[:, market, field], sSigma,
                            LargeSigma, Candles.shape[0] - event_free_data_loss, noPlot=eFreeNoPlot, noLog=eFreeNoLog)
        assert Candles.shape[0] - event_free_data_loss == eventFree.shape[0]
        eventFree = eventFree.astype(Candles.dtype)
        Candles[event_free_data_loss:, market, field] = eventFree

Candles = Candles[event_free_data_loss:]
Times = Times[event_free_data_loss:]
assert Candles.shape[0] == Times.shape[0]

print(Candles.shape, Times.shape)

In [ ]:
Standard = None

if Standardization:
    Candles, Standard = standardize_2(Candles)

In [ ]:
fig = plt.figure(figsize=(16,3))
ax = fig.add_subplot(111)
ax.set_title("Features are custom-standardized" if Standardization else "Features are not standardized")
for market in range(Candles.shape[1]):
    for field in range(Candles.shape[2]):
        ax.plot(Candles[:, market, field], label = "{} @ {}".format(all_field_names[field], all_market_names[market][:-len('USDT')]))
ax.legend(loc = 'upper left')
plt.show()

In [ ]:
sample_anchores_t, sample_anchores_v = get_sample_anchors_2(Candles, Nx, Ny, Ns)
print(sample_anchores_t.shape, sample_anchores_v.shape)

In [ ]:
ds_train, ds_valid, dx, dy = \
get_datasets_2(
    Candles, Time_into_X, Time_into_Y, Times, 
    sample_anchores_t, sample_anchores_v,
    Nx, x_indices, Ny, y_indices, size_time, target_markets,
    BatchSize, shuffle_batch, shuffle=(len(gpus)<=1)
)

In [ ]:
train = ds_train.take(1)
print(train)

In [ ]:
model = None

if mirrored_strategy is None:
    model = build_model_2(
        dx, dy, Num_Layers, Num_Heads, Factor_FF, repComplexity, Dropout_Rate,
        HuberThreshold, Optimizer, Learning_Rate
    )
else:
    with mirrored_strategy.scope():
        model = build_model_2(
            dx, dy, Num_Layers, Num_Heads, Factor_FF, repComplexity, Dropout_Rate,
            HuberThreshold, Optimizer, Learning_Rate
        )

In [ ]:
callbacks = get_callbacks(
    checkpoint_filepath, Checkpoint_Monitor, 
    csvLogger_filepath, 
    EarlyStopping_Min_Monitor, EarlyStopping_Patience
)

try:
    model.load_weights(checkpoint_filepath)
except:
    print("Failed to load a checkpoint")
    pass

In [ ]:
model.fit(
    ds_train, # x and y_true
    validation_data=ds_valid,
    epochs=1, #Epochs_Initial,
    callbacks=callbacks
)

In [ ]:
model.summary()

In [ ]:
model.fit(
    ds_train, # x and y_true
    validation_data=ds_valid,
    epochs=20, #Epochs_Initial,
    callbacks=callbacks
)

In [ ]:
columns = ('loss', 'val_loss', 'mTA', 'val_mTA')
plot_csv_log_history(csvLogger_filepath, columns)

In [ ]:
model.fit(
    ds_train, # x and y_true
    validation_data=ds_valid,
    epochs=50, #Epochs_Initial,
    callbacks=callbacks
)

In [ ]:
columns = ('loss', 'val_loss', 'mTA', 'val_mTA')
plot_csv_log_history(csvLogger_filepath, columns)

In [ ]:
model.fit(
    ds_train, # x and y_true
    validation_data=ds_valid,
    epochs=500, #Epochs_Initial,
    callbacks=callbacks
)

In [ ]:
columns = ('loss', 'val_loss', 'mTA', 'val_mTA')
plot_csv_log_history(csvLogger_filepath, columns)

In [ ]:
model.fit(
    ds_train, # x and y_true
    validation_data=ds_valid,
    epochs=500, #Epochs_Initial,
    callbacks=callbacks
)

In [ ]:
columns = ('loss', 'val_loss', 'mTA', 'val_mTA')
plot_csv_log_history(csvLogger_filepath, columns)